In [1]:
using WaterLily
using LinearAlgebra: norm2
using StaticArrays
using BenchmarkTools

include("TwoD_plots.jl")

vectors (generic function with 1 method)

In [10]:
function block(L=2^5;Re=250,U=1,amp=π/4,ϵ=0.5,thk=2ϵ+√2)
    function sdf(x,t)
        y = x .- SVector(0.,clamp(x[2],-L/2,L/2))
        √sum(abs2,y)-thk/2
    end
    function map(x,t)
        α = amp*cos(t*U/L); R = @SMatrix [cos(α) sin(α); -sin(α) cos(α)]
        R * (x.-SVector(3L-L*sin(t*U/L),4L))
        x+[1,0]*t
    end
    Simulation((6L+2,6L+2),zeros(2),L;U,ν=U*L/Re,body=AutoBody(sdf,map),ϵ)
end

block (generic function with 2 methods)

In [3]:
test() = @benchmark sim_step!(sim,π/4,remeasure=true) setup=(sim=block())

test (generic function with 1 method)

In [11]:
sim_gif!(block();duration=20π,step=π/32,remeasure=true,μbody=true,cfill=:Blues)

tU/L=0.0, Δt=0.25
tU/L=0.0982, Δt=0.48
tU/L=0.1963, Δt=0.477
tU/L=0.2945, Δt=0.475
tU/L=0.3927, Δt=0.478
tU/L=0.4909, Δt=0.481
tU/L=0.589, Δt=0.467
tU/L=0.6872, Δt=0.453
tU/L=0.7854, Δt=0.445
tU/L=0.8836, Δt=0.441
tU/L=0.9817, Δt=0.439
tU/L=1.0799, Δt=0.443
tU/L=1.1781, Δt=0.451
tU/L=1.2763, Δt=0.462
tU/L=1.3744, Δt=0.48
tU/L=1.4726, Δt=0.506
tU/L=1.5708, Δt=0.513
tU/L=1.669, Δt=0.465
tU/L=1.7671, Δt=0.432
tU/L=1.8653, Δt=0.381
tU/L=1.9635, Δt=0.356
tU/L=2.0617, Δt=0.375
tU/L=2.1598, Δt=0.383
tU/L=2.258, Δt=0.358
tU/L=2.3562, Δt=0.355
tU/L=2.4544, Δt=0.378
tU/L=2.5525, Δt=0.427
tU/L=2.6507, Δt=0.457
tU/L=2.7489, Δt=0.476
tU/L=2.8471, Δt=0.473
tU/L=2.9452, Δt=0.474
tU/L=3.0434, Δt=0.478
tU/L=3.1416, Δt=0.485
tU/L=3.2398, Δt=0.48
tU/L=3.3379, Δt=0.475
tU/L=3.4361, Δt=0.475
tU/L=3.5343, Δt=0.479
tU/L=3.6325, Δt=0.482
tU/L=3.7306, Δt=0.445
tU/L=3.8288, Δt=0.426
tU/L=3.927, Δt=0.419
tU/L=4.0252, Δt=0.422
tU/L=4.1233, Δt=0.437
tU/L=4.2215, Δt=0.474
tU/L=4.3197, Δt=0.506
tU/L=4.4179, Δt=0.51


tU/L=35.932, Δt=0.469
tU/L=36.0301, Δt=0.47
tU/L=36.1283, Δt=0.473
tU/L=36.2265, Δt=0.454
tU/L=36.3247, Δt=0.43
tU/L=36.4228, Δt=0.426
tU/L=36.521, Δt=0.435
tU/L=36.6192, Δt=0.452
tU/L=36.7174, Δt=0.479
tU/L=36.8155, Δt=0.507
tU/L=36.9137, Δt=0.49
tU/L=37.0119, Δt=0.476
tU/L=37.1101, Δt=0.471
tU/L=37.2082, Δt=0.456
tU/L=37.3064, Δt=0.471
tU/L=37.4046, Δt=0.473
tU/L=37.5028, Δt=0.474
tU/L=37.6009, Δt=0.478
tU/L=37.6991, Δt=0.486
tU/L=37.7973, Δt=0.48
tU/L=37.8955, Δt=0.475
tU/L=37.9936, Δt=0.475
tU/L=38.0918, Δt=0.457
tU/L=38.19, Δt=0.454
tU/L=38.2882, Δt=0.466
tU/L=38.3863, Δt=0.45
tU/L=38.4845, Δt=0.455
tU/L=38.5827, Δt=0.45
tU/L=38.6809, Δt=0.446
tU/L=38.779, Δt=0.442
tU/L=38.8772, Δt=0.44
tU/L=38.9754, Δt=0.438
tU/L=39.0736, Δt=0.438
tU/L=39.1717, Δt=0.44
tU/L=39.2699, Δt=0.442
tU/L=39.3681, Δt=0.44
tU/L=39.4663, Δt=0.417
tU/L=39.5644, Δt=0.413
tU/L=39.6626, Δt=0.428
tU/L=39.7608, Δt=0.451
tU/L=39.859, Δt=0.479
tU/L=39.9571, Δt=0.496
tU/L=40.0553, Δt=0.466
tU/L=40.1535, Δt=0.458
tU/

┌ Info: Saved animation to 
│   fn = /home/wind/Documents/Proyecto_fluidos_2022/WaterLily/tmp.gif
└ @ Plots /home/wind/.julia/packages/Plots/AJMX6/src/animation.jl:114
